<a href="https://colab.research.google.com/github/cdbm/airbnb-price-rediction-optimizer/blob/main/airbnb_predictorV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd

# Database pre processing

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/cdbm/airbnb-price-rediction-optimizer/main/AB_NYC_2019.csv")
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [4]:
df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [5]:
df = df.drop('name', axis=1)
df = df.drop('host_name', axis=1)
df = df.drop('last_review', axis=1)

In [6]:
df = df[df.price <= 500]

In [7]:
# df.loc[df['price'] <= 50, 'ranges'] = '0_TO_50'
df.loc[df['price'] <= 100, 'ranges'] = '0_TO_100'
df.loc[(df['price'] > 100) & (df['price'] <= 200), 'ranges'] = '101_TO_200'
# df.loc[(df['price'] > 150) & (df['price'] <= 200), 'ranges'] = '151_TO_200'
df.loc[(df['price'] > 200) & (df['price'] <= 300), 'ranges'] = '201_TO_300'
#df.loc[(df['price'] > 250) & (df['price'] <= 300), 'ranges'] = '251_TO_300'
df.loc[(df['price'] > 300) & (df['price'] <= 400), 'ranges'] = '301_TO_400'
#df.loc[(df['price'] > 350) & (df['price'] <= 400), 'ranges'] = '351_TO_400'
df.loc[(df['price'] > 400) & (df['price'] <= 500), 'ranges'] = '401_TO_500'
#df.loc[(df['price'] > 450) & (df['price'] <= 500), 'ranges'] = '551_TO_500'

In [8]:
df['neighbourhood_group'] = df['neighbourhood_group'].astype('category')
df['neighbourhood'] = df['neighbourhood'].astype('category')
df['room_type'] = df['room_type'].astype('category')

In [9]:
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

In [10]:
df['ranges'] = df['ranges'].astype('category')

In [11]:
df['reviews_per_month'].fillna(0, inplace=True) 

In [12]:
df = df.drop('price', axis=1)


In [13]:
df.head()

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,ranges
0,2539,2787,1,107,40.64749,-73.97237,1,1,9,0.21,6,365,101_TO_200
1,2595,2845,2,126,40.75362,-73.98377,0,1,45,0.38,2,355,201_TO_300
2,3647,4632,2,93,40.80902,-73.94190,1,3,0,0.00,1,365,101_TO_200
3,3831,4869,1,41,40.68514,-73.95976,0,1,270,4.64,1,194,0_TO_100
4,5022,7192,2,61,40.79851,-73.94399,0,10,9,0.10,1,0,0_TO_100


In [14]:
df = df.sample(frac=1).reset_index(drop=True)
df = df[:10000]
df.describe()

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.899995e+07,6.768887e+07,1.676900,106.259600,40.728949,-73.951926,0.510500,7.162600,23.582700,1.090646,7.294700,113.193100
std,1.095676e+07,7.826297e+07,0.737401,68.521564,0.054692,0.046054,0.544169,18.488416,45.566558,1.554299,32.574557,131.813304
min,2.595000e+03,2.571000e+03,0.000000,0.000000,40.509430,-74.244420,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,9.519650e+06,7.881532e+06,1.000000,51.000000,40.690798,-73.982845,0.000000,1.000000,1.000000,0.040000,1.000000,0.000000
50%,1.948970e+07,3.091397e+07,2.000000,93.000000,40.722455,-73.955590,0.000000,3.000000,5.000000,0.380000,1.000000,45.000000
75%,2.909846e+07,1.074344e+08,2.000000,177.000000,40.763202,-73.935255,1.000000,5.000000,24.000000,1.610000,2.000000,228.000000
max,3.648409e+07,2.743213e+08,4.000000,218.000000,40.913060,-73.716900,2.000000,500.000000,597.000000,15.320000,327.000000,365.000000


In [15]:
df

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,ranges
0,21440393,8621271,1,51,40.67506,-73.94828,0,3,20,1.00,1,0,101_TO_200
1,31806847,93690639,1,17,40.61109,-74.00239,1,1,0,0.00,1,0,0_TO_100
2,34608937,256726587,2,94,40.75546,-73.99416,0,30,1,0.67,1,176,101_TO_200
3,6651156,34813492,1,19,40.68932,-73.99082,0,2,49,1.00,1,4,101_TO_200
4,17832970,119523037,1,23,40.58186,-73.96020,1,1,2,0.07,4,1,0_TO_100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,36068351,252897042,0,24,40.85247,-73.86418,0,4,2,2.00,1,194,0_TO_100
9996,21212937,76104209,2,200,40.76114,-73.96100,0,30,1,0.05,33,365,101_TO_200
9997,874683,4617159,1,28,40.68777,-73.91601,1,2,22,0.37,1,87,0_TO_100
9998,18805588,5962328,3,77,40.75872,-73.82174,0,30,2,0.19,15,365,101_TO_200


In [16]:
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [17]:
  features = df.drop('ranges',axis=1)
  labels = df['ranges']

In [18]:
feature_train, feature_test, label_train, label_test = train_test_split(features, labels, test_size=0.2, random_state=1)

In [19]:
sc_X = StandardScaler()
feature_train=sc_X.fit_transform(feature_train)
feature_test=sc_X.fit_transform(feature_test)

# Genetic Algorithm

In [20]:
import random

## Parâmetros & Função Fitness


In [21]:
POPULATION_SIZE = 30
GENERATIONS = 100
CHILDREN_RATE = 3

In [22]:
activation = ['identity', 'logistic', 'tanh', 'relu']
solver = ['lbfgs','sgd', 'adam']
learning_rate = ['constant', 'invscaling', 'adaptive']

In [23]:
def calculate_fitness(individue):
  model_mlp = MLPClassifier(activation=individue['activation'],
                            hidden_layer_sizes=individue['hidden_layer_sizes'],
                            learning_rate=individue['learning_rate'],
                            max_iter=individue['max_iter'],
                            solver=individue['solver'],
                            shuffle=individue['shuffle'],
                            warm_start=individue['warm_start'],
                            momentum=individue['momentum'],
                            nesterovs_momentum=individue['nesterovs_momentum'],
                            batch_size=individue['batch_size'])
  

  model_mlp.fit(feature_train, label_train) 

  prediction = model_mlp.predict(feature_test) 
  return accuracy_score(label_test, prediction)
  

## Geração Ppulação 


In [24]:
def generate_individue():
  individue = {
      'activation':  activation[random.randint(0, 3)],
      'hidden_layer_sizes' : (random.randint(100, 200), random.randint(50, 100) ,random.randint(10, 50)),
      'learning_rate': learning_rate[random.randint(0,2)],
      'max_iter' : random.randint(100, 250),
      'solver': solver[random.randint(0,2)],
      'shuffle' : (random.randint(0,1) == 1),
      'warm_start' : (random.randint(0,1) == 1),
      'momentum' : random.randrange(0,1),
      'nesterovs_momentum' : (random.randint(0,1) == 1),
      'batch_size' : random.randint(100,300),
      'fitness': 0  
  }

  individue['fitness'] = calculate_fitness(individue)
  return individue

In [25]:
def sortByFitness(e):
  return e['fitness']

In [26]:
def initiate_population(population_size):
  population = [[]] * population_size
  population = [generate_individue() for i in range(0, population_size)]
  population.sort(key=sortByFitness, reverse=True)
  return population

## Mutações


In [27]:
def choose_parents(population):
  father = random.randint(0, POPULATION_SIZE - 1)
  mother = random.randint(0, POPULATION_SIZE - 1)
  return [population[father], population[mother]]

In [28]:
def choose_candidates(population):
  candidatesIndex = []
  for x in range(0, round(POPULATION_SIZE * 0.3) ):
    index = random.randint(0, POPULATION_SIZE - 1)
    while index in candidatesIndex:
      index = random.randint(0, POPULATION_SIZE - 1)
    candidatesIndex.append(index)

  candidates = []
  for x in candidatesIndex:
    candidates.append(population[x])
  return candidates   

def choose_parents_by_roullete(population):
  population.sort(key=sortByFitness, reverse=True)
  candidates = population[:round(POPULATION_SIZE / 2)]
  probabilities = [];
  sumOfFitness = 0;
  previousProbabilities = 0;
  parents = [];

  for i in candidates:
    sumOfFitness = sumOfFitness + i['fitness']

  candidates.sort(key=sortByFitness)

 
  for cand in candidates:
    prob = previousProbabilities + (cand['fitness'] / sumOfFitness);
    previousProbabilities = prob;
    probabilities.append(prob);
  
  while len(parents)  < 2:
    number = random.randint(1, 10) /10
    found = False
    for y in range(0, len(probabilities)):
      if (number <= probabilities[y]) & (not found):
        parents.append(candidates[y])
        found = True
  
  return parents

In [29]:
def mutate_activation(individue):
  new_attribute = activation[random.randint(0,3)] 
  while new_attribute == individue['activation']:
    new_attribute = activation[random.randint(0,3)] 
  individue['activation'] = new_attribute
  return individue

def mutate_learning_rate(individue):
  new_attribute = learning_rate[random.randint(0,2)] 
  while new_attribute == individue['learning_rate']:
    new_attribute = learning_rate[random.randint(0,2)] 
  individue['learning_rate'] = new_attribute
  return individue

def mutate_solver(individue):
  new_attribute = solver[random.randint(0,2)] 
  while new_attribute == individue['solver']:
    new_attribute = solver[random.randint(0,2)] 
  individue['solver'] = new_attribute
  return individue

def mutate_momentum(individue):
  new_attribute = momentum[random.randint(0,1)] 
  while new_attribute == individue['momentum']:
    new_attribute = momentum[random.randint(0,1)] 
  individue['momentum'] = new_attribute
  return individue

def mutate_batch_size(individue):
  new_attribute = batch_size[random.randint(100,300)] 
  while new_attribute == individue['batch_size']:
    new_attribute = batch_size[random.randint(100,300)] 
  individue['batch_size'] = new_attribute
  return individue

def mutate_shuffle(individue):#Para booleanos, deixar full sorte
  new_attribute = solver[random.randint(0,1) == 1] 
  individue['shuffle'] = new_attribute
  return individue

def mutate_warm_start(individue):
  new_attribute = warm_start[random.randint(0,1) == 1] 
  individue['warm_start'] = new_attribute
  return individue

def mutate_nesterovs_momentum(individue):
  new_attribute = nesterovs_momentum[random.randint(0,1) == 1] 
  individue['nesterovs_momentum'] = new_attribute
  return individue

# 'shuffle' : (random.randint(0,1) == 1),
#       'warm_start' : (random.randint(0,1) == 1),
#       'momentum' : random.randrange(0,1),
#       'nesterovs_momentum' : (random.randint(0,1) == 1),

def mutate(individue):
  mutations = [mutate_activation, mutate_learning_rate, mutate_solver,mutate_momentum,mutate_shuffle,mutate_warm_start,mutate_nesterovs_momentum,mutate_batch_size]
  prob = random.randint(0, 10)
  if prob >= 6:
    mutation1 = random.randint(0,len(mutations))
    mutation2 = random.randint(0,len(mutations))

    individue = mutations[mutation1](individue)
    individue = mutations[mutation2](individue)

  return individue

In [30]:
def make_children(population):
  parents = choose_parents_by_roullete(population)
  
  for x in range(0, 2):
    child = {'activation': parents[random.randint(0,1)]['activation'],
            'hidden_layer_sizes': (parents[random.randint(0,1)]['hidden_layer_sizes'][0],parents[random.randint(0,1)]['hidden_layer_sizes'][1], parents[random.randint(0,1)]['hidden_layer_sizes'][2]),
            'learning_rate': parents[random.randint(0,1)]['learning_rate'],
            'max_iter': parents[random.randint(0,1)]['max_iter'],
            'solver': parents[random.randint(0,1)]['solver'],
            'shuffle': parents[random.randint(0,1)]['shuffle'],
            'warm_start': parents[random.randint(0,1)]['warm_start'],
            'momentum' : parents[random.randint(0,1)]['momentum'],
            'nesterovs_momentum' : parents[random.randint(0,1)]['nesterovs_momentum'],
            'batch_size' : parents[random.randint(0,1)]['batch_size'],
            'fitness': 0}
    

    child = mutate(child)
    child['fitness'] = calculate_fitness(child)
    population.append(child)
  
  population.sort(key=sortByFitness, reverse=True)
  return population[:POPULATION_SIZE]

In [31]:
population = initiate_population(POPULATION_SIZE)
population.sort(key=sortByFitness, reverse=True)
print(population)
for x in range(0, GENERATIONS):
  print("generation: ")
  print(x)
  population = make_children(population)
  print(population)

print("Better was: ") 
print(population[0])
print("With a accuracy of: ")
print(population[0]['fitness'])

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)

KeyboardInterrupt: ignored